## G-CARE Benchmark

In [2]:
using Graphs: DiGraph
using StatsPlots
include("Source/datasets.jl")
include("Source/QuasiStableCardinalityEstimator.jl")

get_exact_size (generic function with 1 method)

In [18]:
aids_data_file_path = "dataset/aids/aids.txt"
human_data_file_path = "dataset/human/human.txt"
lubm80_data_file_path = "dataset/lubm80/lubm80.txt"
yago_data_file_path = "dataset/yago/yago.txt"

human_data = load_dataset(human_data_file_path)
aids_data = load_dataset(aids_data_file_path)
#lubm80_data = load_dataset(lubm80_data_file_path)
#yago_data = load_dataset(yago_data_file_path)
nothing

In [19]:
#datasets = [aids_data, human_data, lubm80_data, yago_data]
#dataset_names = ["aids", "human", "lubm80", "yago"]
datasets = [human_data, aids_data]
dataset_names = ["human", "aids"]

2-element Vector{String}:
 "human"
 "aids"

In [26]:
build_time = []
summary_size = []
color_summaries = []
for dataset in datasets
    results = @timed generate_color_summary(dataset, 64, verbose=true)
    push!(build_time, results[2])
    push!(summary_size,  get_color_summary_size(results[1]))
    push!(color_summaries, results[1])
end

Started coloring
Finished coloring
Started bloom filters
Finished bloom filters
Started cardinality counts
Finished cardinality counts
Started tracking statistics
Finished tracking statistics
Started coloring
Finished coloring
Started bloom filters
Finished bloom filters
Started cardinality counts
Finished cardinality counts
Started tracking statistics
Finished tracking statistics


In [31]:
human_query_directories = ["/queryset/human/Chain_3/",
"/queryset/human/Graph_3/",
"/queryset/human/Star_3/",
"/queryset/human/Tree_3/",
                    ]
human_query_paths = [readdir(pwd() * dir, join=true) for dir in human_query_directories]
human_query_paths = [(human_query_paths...)...]
human_exact_sizes = []
human_bounds = []
human_relative_errors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in human_query_paths
    println("Query: ", query_path)
    id_and_query = load_query(query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality(replace(query_path, "queryset"=>"TrueCardinalities"))
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./exact_size_results[1])
    bound_results[1][2] = max(1, bound_results[1][2])
    push!(human_exact_sizes, gcare_size)
    push!(human_bounds, bound_results[1])
    push!(human_relative_errors, bound_results[1] ./ gcare_size)
end

Summary Size: 359196
Summary Build Time: 2.663009922
Query: /home/kylebd99/Cardinality-with-Colors/queryset/human/Chain_3/uf_Q_1_1.txt
Bound Time: 0.001460951
Exact Size Time: 0.000663035
Exact Size: 72
GCare Exact Size: 72
Bound [Lower, Avg, Upper]: [0.0, 104.79770915583273, 71313.0]
Relative Error [Lower, Avg, Upper]: [0.0, 1.4555237382754547, 990.4583333333334]
Query: /home/kylebd99/Cardinality-with-Colors/queryset/human/Chain_3/uf_Q_1_2.txt
Bound Time: 0.001135692
Exact Size Time: 0.286747285
Exact Size: 22
GCare Exact Size: 22
Bound [Lower, Avg, Upper]: [0.0, 2875.0730584667353, 329418.0]
Relative Error [Lower, Avg, Upper]: [0.0, 130.68513902121524, 14973.545454545454]
Query: /home/kylebd99/Cardinality-with-Colors/queryset/human/Chain_3/uf_Q_1_3.txt
Bound Time: 0.001052217
Exact Size Time: 0.000552107
Exact Size: 10
GCare Exact Size: 10
Bound [Lower, Avg, Upper]: [0.0, 25.442562929061786, 25072.0]
Relative Error [Lower, Avg, Upper]: [0.0, 2.5442562929061787, 2507.2]
Query: /home/k

In [32]:
using Statistics
println(maximum([log10(x[2]) for x in human_relative_errors]))
println(mean([log10(x[2]) for x in human_relative_errors]))
println(minimum([log10(x[2]) for x in human_relative_errors]))

2.1162262041985125
-0.09453409140278614
-1.5314789170422551


In [29]:
aids_query_directories = ["/queryset/aids/Chain_3/",
"/queryset/aids/Chain_6/",
"/queryset/aids/Chain_9/",
"/queryset/aids/Chain_12/",
"/queryset/aids/Cycle_3/",
"/queryset/aids/Cycle_6/",
"/queryset/aids/Flower_6/",
"/queryset/aids/Flower_9/",
"/queryset/aids/Flower_12/",
"/queryset/aids/Graph_3/",
"/queryset/aids/Graph_6/",
"/queryset/aids/Graph_9/",
"/queryset/aids/Graph_12/",
"/queryset/aids/Petal_6/",
"/queryset/aids/Petal_9/",
"/queryset/aids/Petal_12/",
"/queryset/aids/Star_3/",
"/queryset/aids/Star_6/",
"/queryset/aids/Star_9/",
"/queryset/aids/Tree_3/",
"/queryset/aids/Tree_6/",
"/queryset/aids/Tree_9/",
"/queryset/aids/Tree_12/"]
aids_query_paths = [readdir(pwd() * dir, join=true) for dir in aids_query_directories]
aids_query_paths = [(aids_query_paths...)...]
aids_exact_sizes = []
aids_bounds = []
aids_relative_errors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in aids_query_paths
    println("Query: ", query_path)
    id_and_query = load_query(query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[2])
    gcare_size = load_true_cardinality(replace(query_path, "queryset"=>"TrueCardinalities"))
    println("Bound Time: ", bound_results[2])
    println("Exact Size: ", gcare_size)
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", bound_results[1]./gcare_size)
    bound_results[1][2] = max(1, bound_results[1][2])
    push!(aids_exact_sizes, gcare_size)
    push!(aids_bounds, bound_results[1])
    push!(aids_relative_errors, bound_results[1] ./ gcare_size)
end

Summary Size: 359196
Summary Build Time: 2.663009922
Query: /home/kylebd99/Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_0_2.txt
Bound Time: 0.000512285
Exact Size: 8
Bound [Lower, Avg, Upper]: [0.0, 3.0253029331868264, 88272.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.3781628666483533, 11034.0]
Query: /home/kylebd99/Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_1.txt
Bound Time: 0.000469355
Exact Size: 12
Bound [Lower, Avg, Upper]: [0.0, 6.626766347820973, 178119.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.5522305289850811, 14843.25]
Query: /home/kylebd99/Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_2.txt
Bound Time: 0.000450099
Exact Size: 14
Bound [Lower, Avg, Upper]: [0.0, 10.71722139616455, 273927.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.7655158140117536, 19566.214285714286]
Query: /home/kylebd99/Cardinality-with-Colors/queryset/aids/Chain_3/uf_Q_1_3.txt
Bound Time: 0.000471642
Exact Size: 39
Bound [Lower, Avg, Upper]: [0.0, 23.24952349097013, 1.371

In [30]:
using Statistics
println(maximum([log10(x[2]) for x in aids_relative_errors]))
println(mean([log10(x[2]) for x in aids_relative_errors]))
println(minimum([log10(x[2]) for x in aids_relative_errors]))

2.6686015167651913
-1.3494790192246389
-5.504089068119823


In [ ]:
lubm80_query_paths = ["lubm80/lubm80_Q2.txt",
# "lubm80/lubm80_Q4.txt",
# "lubm80/lubm80_Q7.txt",
# "lubm80/lubm80_Q8.txt",
# "lubm80/lubm80_Q9.txt",
# "lubm80/lubm80_Q12.txt"
                    ]
exactSizes = []
bounds = []
relativeErrors = []
println("Summary Size: ", summary_size[1])
println("Summary Build Time: ", build_time[1])
for query_path in lubm80_query_paths
    println("Query: ", query_path)
    id_and_query = load_query("queryset/" * query_path)
    id = id_and_query[1]
    query = id_and_query[2]
    bound_results = @timed get_cardinality_bounds(query, color_summaries[1])
    exact_size_results = @timed get_exact_size(query, datasets[1])
    gcare_size = load_true_cardinality("TrueCardinalities/" * query_path)
    println("Bound Time: ", bound_results[2])
    println("Exact Size Time: ", exact_size_results[2])
    println("Exact Size: ", exact_size_results[1])
    println("GCare Exact Size: ", gcare_size)
    if gcare_size != exact_size_results[1]
        println("Bad Exact Size Estimate!!")
        break
    end
    println("Bound [Lower, Avg, Upper]: ", bound_results[1])
    println("Relative Error [Lower, Avg, Upper]: ", (bound_results[1])./exact_size_results[1])
    push!(exactSizes, gcare_size)
    push!(bounds, bound_results[1])
    push!(relativeErrors, bound_results[1] ./ gcare_size)
end

Summary Size: 7209
Summary Build Time: 599.4344749


Query: lubm80/lubm80_Q2.txt


Bound Time: 0.2822331
Exact Size Time: 28.6690044
Exact Size: 212
GCare Exact Size: 212
Bound [Lower, Avg, Upper]: [0.0, 0.0, 432014.0]
Relative Error [Lower, Avg, Upper]: [0.0, 0.0, 2037.801886792453]


In [ ]:
exactSizes